In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import cv2

In [ ]:
path = 'D:\\IIM Course\\Capstone Project\\Project/'
os.listdir(path)

In [ ]:
train_df = pd.read_csv(path + 'train.csv')
train_df.head()

In [ ]:
# Get hot-encoded labels for targets
df2 = pd.get_dummies(train_df, columns=['target'])
display(df2)

In [ ]:
# Separating the hot-encoded columns
df1 = df2.drop(['files', 'images'],axis=1)
display(df1)

In [ ]:
# Merge with the original dataset
df = train_df.merge(df1, left_index = True, right_index = True)
display(df)

In [ ]:
# Drop non-essential column
df = df.drop(['files'],axis=1)
display(df)

In [ ]:
# Check for null items
df.isnull().sum().sum()

In [ ]:
# Plot images wrt the given targets
plt.figure(figsize=(10, 10))

for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    img = mpimg.imread(path + '/fashion/' + df["images"][i])
    img = cv2.resize(img, (224, 224))
    plt.imshow(img)
    plt.title(df["target"][i])
    plt.axis("off")
    
    X = np.array(df)

In [ ]:
# Check names of the columns
train_df.columns

In [ ]:
# Check names of the new dataframe columns
df.columns

In [ ]:
# Check unique columns
df['target'].unique()

In [ ]:
# Countplot to check if data has equal shape
sns.countplot(df['target'])

In [ ]:
# Count to check if data has equal sape
df["target"].value_counts()

In [ ]:
from tqdm import tqdm
from keras.preprocessing import image


In [ ]:
# Loading and Preprocessing the data
train_image = []
for i in tqdm(range(df.shape[0])):
    img = image.load_img(path + '/fashion/' +df['images'][i],target_size=(256,256,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

In [ ]:
# Shape of the array
X.shape

In [ ]:
# Plot and visualize preprocessed image
plt.imshow(X[2])

In [ ]:
# Plot and visualize preprocessed image
df['target'][2]

In [ ]:
y = np.array(df.drop(['target', 'images'],axis=1))
y.shape

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [ ]:
# Randomly separate 10% of the images as our validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
# Define the architecture of our model
model = Sequential()
model.add(Conv2D(16, (3, 3), strides = 1, padding = "same", activation = "relu", input_shape=(224,224,3)))
model.add(Dropout(0.25))                 
model.add(Conv2D(16, (3, 3), strides = 1, padding = "same", activation = "relu"))                 
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.4))
model.add(Conv2D(64, (3, 3), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.4))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='sigmoid'))

In [ ]:
from keras.models import load_model
from keras.models import model_from_json
import json

In [ ]:
# load json and create model
json_file = open('model_resized.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_resized.h5")
print("Loaded model from disk")


In [ ]:
# summarize history for accuracy
plt.plot(loaded_model.loaded_model['accuracy'])
plt.plot(loaded_model.loaded_model['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(loaded_model.loaded_model['loss'])
plt.plot(loaded_model.loaded_model['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# evaluate the test set
scores = loaded_model.evaluate(X_test, y_test, verbose=0)
print("Final test %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# evaluate the train set
scores = loaded_model.evaluate(X_train, y_train, verbose=0)
print("Final train %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [ ]:
data = json.loads(loaded_model_json.read())
data

In [ ]:
# Summary of the model
model.summary()

In [ ]:
# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Training the model.
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64)

In [ ]:
# Save model
model.save("modelVGG.h5")
print("Saved model to disk")

In [ ]:
# load model
#model = load_model('model.h5')

In [ ]:
# evaluate the test set
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# evaluate the train set
scores = model.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
img = image.load_img(path + '/predictImages/' +'bag3.jpg',target_size=(224,224,3))
img = image.img_to_array(img)
img = img/255

In [ ]:
classes = np.array(df.columns[2:])
proba = model.predict(img.reshape(1,224,224,3))
among4 = np.argsort(proba[0])[:-5:-1]
for i in range(4):
    print("{}".format(classes[among4[i]])+" ({:.3})".format(proba[0][among4[i]]))
plt.imshow(img)